In [264]:
import ast
import pandas as pd
import numpy as np
import torch
from tensorflow import keras
from tdc.single_pred.adme import ADME
from tdc import Evaluator
from tqdm.notebook import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader
from rdkit import Chem
from rdkit.Chem import AllChem
from matplotlib import pyplot as plt
from IPython import display
from rdkit.Chem import BondType
from rdkit.Chem.Draw import MolsToGridImage
from torch import tensor

from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.molSize = 250,250



In [289]:
csv_path = keras.utils.get_file(
    "250k_rndm_zinc_drugs_clean_3.csv",
    "https://raw.githubusercontent.com/aspuru-guzik-group/chemical_vae/master/models/zinc_properties/250k_rndm_zinc_drugs_clean_3.csv",
)

df = pd.read_csv("~/.keras/datasets/250k_rndm_zinc_drugs_clean_3.csv")
df["smiles"] = df["smiles"].apply(lambda s: s.replace("\n", ""))
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249455 entries, 0 to 249454
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   smiles  249455 non-null  object 
 1   logP    249455 non-null  float64
 2   qed     249455 non-null  float64
 3   SAS     249455 non-null  float64
dtypes: float64(3), object(1)
memory usage: 7.6+ MB


In [290]:
SMILE_CHARSET = '["C", "B", "F", "I", "H", "O", "N", "S", "P", "Cl", "Br", None]'

bond_mapping = {"SINGLE": 0, "DOUBLE": 1, "TRIPLE": 2, "AROMATIC": 3, "SELF": 4}
bond_mapping.update(
    {0: BondType.SINGLE, 1: BondType.DOUBLE, 2: BondType.TRIPLE, 3: BondType.AROMATIC, 4: "None"}
)
SMILE_CHARSET = ast.literal_eval(SMILE_CHARSET)
MAX_BONDNUM = 150
MAX_MOLSIZE = 64 #max(df["smiles"].str.len())
SMILE_to_index = dict((c, i) for i, c in enumerate(SMILE_CHARSET))
index_to_SMILE = dict((i, c) for i, c in enumerate(SMILE_CHARSET))
atom_mapping = dict(SMILE_to_index)
atom_mapping.update(index_to_SMILE)

BATCH_SIZE = 32
EPOCHS = 500

VAE_LR = 5e-4
NUM_ATOMS = 32  # Maximum number of atoms

ATOM_DIM = len(SMILE_CHARSET)  # Number of atom types
BOND_DIM = 5  # Number of bond types
LATENT_DIM = 128  # Size of the latent space

In [345]:
def molecule_to_graph(molecule):
    # Converts SMILES to molecule object
    
    mol_atoms = molecule.GetNumAtoms()    # Initialize adjacency and feature tensor
    num_of_bonds = molecule.GetNumBonds()
    edge_index = [[],[]]
    edge_features = []
    adj_mat = np.zeros((NUM_ATOMS, ATOM_DIM))
    
    features = np.zeros((NUM_ATOMS, ATOM_DIM))

    # loop over each atom in molecule
    for atom in molecule.GetAtoms():
        atom_idx = atom.GetIdx()
        atom_type = atom_mapping[atom.GetSymbol()]
        atomic_num = atom.GetAtomicNum()
        degree = atom.GetDegree()
        #mass = atom.GetMass()
        # edge_index[0].append(atom_idx)
        # edge_index[1].append(atom_idx)
        # edge_embbeding = list(np.eye(BOND_DIM)[0])
        # edge_features.append(edge_embbeding)
        #chem_features = [atomic_num, degree] #, mass]
        #print(atom_idx)
        features[atom_idx] = np.eye(ATOM_DIM)[atom_type] #np.concatenate((np.eye(ATOM_DIM)[atom_type][:-2], chem_features), axis=0)
    for i in range(mol_atoms, NUM_ATOMS):
        features[i][ATOM_DIM-1] = 1
        edge_index[0].append(i)
        edge_index[1].append(i)
        edge_embbeding = list(np.eye(BOND_DIM)[BOND_DIM-1])
        edge_features.append(edge_embbeding)
        # loop over one-hop neighbors
    for bond in molecule.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        bond_type_idx = bond_mapping[bond.GetBondType().name]
        edge_index[0] = list(np.append(edge_index[0], [i, j]))
        edge_index[1] = list(np.append(edge_index[1], [j, i]))
        edge_embbeding = list(np.eye(BOND_DIM)[bond_type_idx])
        edge_features.append(edge_embbeding)
        edge_features.append(edge_embbeding)

    
    return tensor(features), tensor(edge_index), tensor(edge_features)


def graph_to_molecule(graph):  # for edges in adjacency matrix format
    features, adjacency, edge_features = graph
    #print(features, adjacency, edge_features)
    #print("*")
    if adjacency.shape[0] == 2:
       adjacency, edge_features = edge_idx_to_adj(adjacency, edge_features)
    #print(features, adjacency, edge_features)
    # RWMol is a molecule object intended to be edited
    molecule = Chem.RWMol()

    for i in range(len(adjacency)):
        adjacency[i][i] = 0

    # Remove "no atoms" & atoms with no bonds
    mol_idx_to_graph_idx = np.where(
        (np.argmax(features, axis=1) != ATOM_DIM - 1)
        & (adjacency.sum(axis=1) != 0)
    )
    mol_idx_to_graph_idx = np.squeeze(mol_idx_to_graph_idx)

    graph_idx_to_mol_idx = np.full_like([], fill_value=-1, shape=NUM_ATOMS, dtype=int)
    for mol_idx, graph_idx in enumerate(mol_idx_to_graph_idx):
        graph_idx_to_mol_idx[graph_idx] = mol_idx
    
    for mol_idx, graph_idx in enumerate(mol_idx_to_graph_idx):
        #print(graph_idx)
        #print(features)
        atom_type_idx = int(torch.argmax(features[graph_idx]))
        #print(atom_type_idx)
        atom = Chem.Atom(atom_mapping[atom_type_idx])
        _ = molecule.AddAtom(atom)

    for mol_idx_1, graph_idx_1 in enumerate(mol_idx_to_graph_idx):
        for mol_idx_2, graph_idx_2 in enumerate(mol_idx_to_graph_idx):
            if adjacency[graph_idx_1][graph_idx_2] == 1 and graph_idx_2 > graph_idx_1:
                #print(edge_features[graph_idx_1][graph_idx_2])
                edge_type_encoded = int(np.argmax(edge_features[graph_idx_1][graph_idx_2]))

                #print(edge_type_encoded)
                bond_type = bond_mapping[edge_type_encoded]       # TODO
                molecule.AddBond(int(mol_idx_1), int(mol_idx_2), bond_type)

    flag = Chem.SanitizeMol(molecule, catchErrors=True)

    if flag != Chem.SanitizeFlags.SANITIZE_NONE:
        return None
    #print("OK")
    return molecule

def adj_to_index(adj_matrix, attr_matrix):
    edge_index=[[], []]
    edge_attr = []
    for i, row in enumerate(adj_matrix):
        adj_matrix[i][i]=0 
        for j in range(len(row)):
            if row[j] == 1:
                edge_index[0].append(i)
                #edge_index[0].append(j)
                #edge_index[1].append(i)
                edge_index[1].append(j)
                edge_embedding = attr_matrix[i][j]
                #edge_type_encoded = np.argmax(edge_embedding)
                edge_attr.append(edge_embedding)
    return torch.tensor(edge_index), torch.tensor(edge_attr)


def edge_idx_to_adj(edge_index, edges_attr):
    #print(edges_attr)
    adj = np.zeros((NUM_ATOMS, NUM_ATOMS))
    attr_matrix = np.zeros((NUM_ATOMS, NUM_ATOMS, BOND_DIM))
    for i in range(edge_index.shape[1]):
        k = edge_index[0][i]
        j = edge_index[1][i]
        adj[k][j] = 1
        attr_matrix[k][j] = edges_attr[i]
    return torch.tensor(adj), torch.tensor(attr_matrix)

def batch_edge_idx_to_adjacency(edge_index, edges_attr, batch):
    adj = np.zeros((BATCH_SIZE*NUM_ATOMS, BATCH_SIZE*NUM_ATOMS))
    attr_matrix = np.zeros((BATCH_SIZE*NUM_ATOMS,BATCH_SIZE*NUM_ATOMS, BOND_DIM))
    adj_reduced = np.zeros((0, NUM_ATOMS))
    attr_reduced = np.zeros((0, NUM_ATOMS, BOND_DIM))
    for i in range(edge_index.shape[1]):
        k = edge_index[0][i]
        j = edge_index[1][i]
        adj[k][j] = 1
        attr_matrix[k][j] = edges_attr[i]
    for i in range(BATCH_SIZE):
        #print(adj.shape, (adj[i*NUM_ATOMS:(i+1)*NUM_ATOMS,i*NUM_ATOMS:(i+1)*NUM_ATOMS]).shape)
        adj_reduced = np.concatenate([adj_reduced, adj[i*NUM_ATOMS:(i+1)*NUM_ATOMS,i*NUM_ATOMS:(i+1)*NUM_ATOMS]], axis=0)
        attr_reduced = np.concatenate([attr_reduced, attr_matrix[i*NUM_ATOMS:(i+1)*NUM_ATOMS,i*NUM_ATOMS:(i+1)*NUM_ATOMS,:]], axis=0)
    return torch.tensor(adj_reduced), torch.tensor(attr_reduced)


In [326]:
z = np.array([[1,2,3]])
y = np.array([[4,5,6]])
np.concatenate([z, y], axis = 0)

array([[1, 2, 3],
       [4, 5, 6]])

In [293]:
from sklearn.discriminant_analysis import StandardScaler
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


class Featurizer():
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __call__(self, df):
        graphs = []
        labels = []
        for i, row in df.iterrows():
            smiles = row["smiles"]
            molecule = Chem.MolFromSmiles(smiles)
            if molecule.GetNumAtoms() < 32:
                features, edge_index, edge_attr = molecule_to_graph(molecule) 
                graphs.append((features, edge_index, edge_attr))
                labels.append(molecule)
            
        return [Data(
            features=features.float(), 
            edge_index=edge_index.int(), 
            edge_attr=edge_attr.float(),
            mol=labels)
            for ((features, edge_index, edge_attr), labels) in zip(graphs, labels)]

In [294]:
import sklearn
from sklearn import model_selection
from torch.utils.data import DataLoader

train_data, test_data = model_selection.train_test_split(df, test_size=0.2)

In [295]:
featurizer = Featurizer()
data_train = featurizer(train_data[:1024])
data_val = featurizer(test_data[:128])

from torch_geometric.loader import DataLoader
train_loader = DataLoader(data_train, batch_size=BATCH_SIZE)
val_loader = DataLoader(data_val, batch_size=BATCH_SIZE)

In [316]:
loop = tqdm(enumerate(train_loader))
for i, data in loop:
    if i > 0: break
    print(data)
    print(data.edge_index[0][100:120])
    print(data.edge_index[1][100:120])
    print(data.edge_attr)
    #print(edge_idx_to_adj(data.edge_index).sum(axis=1))
    #mol = graph_to_molecule((data.features, data.edge_index, data.edge_attr))

0it [00:00, ?it/s]

DataBatch(edge_index=[2, 1866], edge_attr=[1866, 5], features=[1024, 12], mol=[32], batch=[1024], ptr=[33])
tensor([54, 54, 55, 55, 56, 56, 57, 48, 39, 57, 49, 47, 42, 56, 51, 88, 89, 90,
        91, 92], dtype=torch.int32)
tensor([53, 55, 54, 56, 55, 57, 56, 39, 48, 49, 57, 42, 47, 51, 56, 88, 89, 90,
        91, 92], dtype=torch.int32)
tensor([[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        ...,
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.]])


In [ ]:
mol

In [ ]:
data_train[0].features

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0

In [378]:
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool
import torch.nn.functional as F
from torch.nn.functional import dropout, one_hot
import torch


HIDDEN_SIZE=512

class Encoder(torch.nn.Module):
    def __init__(self, hidden_size=HIDDEN_SIZE, atom_dim = ATOM_DIM, latent_dim=LATENT_DIM):
        super(Encoder, self).__init__()
        torch.manual_seed(12345)

        self.conv = [
            GATConv(atom_dim, hidden_size),
            GATConv(hidden_size, hidden_size),
            GATConv(hidden_size, hidden_size),
            GATConv(hidden_size, hidden_size),
            GATConv(hidden_size, hidden_size),
            GATConv(hidden_size, hidden_size)
        ]
        self.pooling = global_mean_pool
        self.dense = [
            torch.nn.Linear(in_features=hidden_size, out_features=hidden_size)
        ]
        self.mean = torch.nn.Linear(in_features=hidden_size, out_features=latent_dim)
        self.std = torch.nn.Linear(in_features=hidden_size, out_features=latent_dim)



    def forward(self, x, edge_index, edge_attr, batch): 
        for conv_layer in self.conv:
            x = conv_layer(x, edge_index, edge_attr)
            x = torch.nn.ELU()(x)
        print(batch)
        print("before pooling ", x.shape)
        x = self.pooling(x, batch=batch) #, size = x.shape[1])#[0]
        print("after pooling ", x.shape)
        #x = torch.flatten(x)
       # print("after flattening ", x.shape)
        for layer in self.dense:
            x = layer(x)
            x = torch.nn.ELU()(x)
        print("after densing ", x.shape)
        mu, std = self.mean(x), self.std(x)

        return mu, std
    
    def loss():
        pass

class Decoder(torch.nn.Module):
    def __init__(self, hidden_size=HIDDEN_SIZE, atom_dim = ATOM_DIM, latent_dim=LATENT_DIM) -> None:
        super(Decoder, self).__init__()
        torch.manual_seed(12345)
        self.edges_from_latent = [
            torch.nn.Linear(in_features=latent_dim, out_features=int(NUM_ATOMS)*int(NUM_ATOMS)),
            torch.nn.Linear(in_features=int(NUM_ATOMS)*int(NUM_ATOMS), out_features=int(NUM_ATOMS)*int(NUM_ATOMS))
        ]

        self.edges_types_from_latent = [
            torch.nn.Linear(in_features=latent_dim, out_features=int(NUM_ATOMS)*int(NUM_ATOMS)*5),
            torch.nn.Linear(in_features=int(NUM_ATOMS)*int(NUM_ATOMS)*5, out_features=int(NUM_ATOMS)*int(NUM_ATOMS)*5),
            torch.nn.Linear(in_features=int(NUM_ATOMS)*int(NUM_ATOMS)*5, out_features=int(NUM_ATOMS)*int(NUM_ATOMS)*5)
        ]

        self.nodes_from_latent = [
            torch.nn.Linear(in_features=latent_dim, out_features=int(NUM_ATOMS)*int(ATOM_DIM)),
            torch.nn.Linear(in_features=int(NUM_ATOMS)*int(ATOM_DIM), out_features=int(NUM_ATOMS)*int(ATOM_DIM))
        ]

    def forward(self, z_latent):
        edges = torch.clone(z_latent)
        for layer in self.edges_from_latent:
            edges = layer(edges)
        edges = torch.reshape(edges, (NUM_ATOMS, NUM_ATOMS)) + torch.transpose_copy(torch.reshape(edges, (NUM_ATOMS, NUM_ATOMS)), 0, 1)
        edges = torch.sigmoid(edges)


        edges_attr = torch.clone(z_latent)
        for layer in self.edges_types_from_latent:
            edges_attr = layer(edges_attr)
        edges_attr = torch.reshape(edges_attr, (NUM_ATOMS, NUM_ATOMS, 5)) + torch.transpose_copy(torch.reshape(edges_attr, (NUM_ATOMS, NUM_ATOMS, 5)), 0, 1)
        edges_attr = torch.tensor(edges_attr, dtype=float)
        edges_attr = torch.softmax(edges_attr, dim= 1)
        
        nodes = torch.clone(z_latent)
        for layer in self.nodes_from_latent:
            nodes = layer(nodes)
        nodes = torch.reshape(nodes, (NUM_ATOMS, ATOM_DIM))
        nodes = torch.tensor(nodes, dtype=float)
        nodes = torch.softmax(nodes, dim= 1)
        return nodes, edges, edges_attr

    def loss():
        pass

 

class GraphNeuralNetwork(torch.nn.Module):  # TODO: assign hyperparameters to attributes and define the forward pass
    def __init__(self, encoder, decoder, hidden_size, atom_dim = ATOM_DIM, latent_dim=LATENT_DIM):
        super(GraphNeuralNetwork, self).__init__()
        torch.manual_seed(12345)

        self.encoder = Encoder(hidden_size)
        self.decoder = Decoder(hidden_size)

        #decode edges:
        
    
    def encode(self, features, edge_index, edge_attr, batch):
        return self.encoder(features, edge_index, edge_attr, batch)

    def reparametrize(self, mu, std):
        eps = torch.randn_like(std)
        z_reparametrized = mu + std*eps
        return z_reparametrized
        

    def decode(self, z_latent):
        return self.decoder(z_latent)

    def forward(self, features, adj, edge_attr, batch):
        mu, std = self.encode(features, adj, edge_attr, batch)
        z = self.reparametrize(mu, std)
        nodes_dec, edges_dec, edges_attr_dec = self.decode(z)

        return nodes_dec, edges_dec, edges_attr_dec
    

    # def reconstruction_loss(self, nodes_generated, nodes_real, edges_generated, edges_real, edges_attr_generated, edges_attr_real):
    #     loss = torch.nn.CrossEntropyLoss()
    #     nodes_loss = loss(nodes_generated, nodes_real)
    #     edges_loss = loss(edges_generated, edge_idx_to_adj(edges_real))
    #     edges_attr_loss = loss(edges_attr_generated, edges_attr_real)
    #     return nodes_loss + edges_loss + edges_attr_loss

    def eval(self, test_data_loader):
        loss_fn = torch.nn.CrossEntropyLoss()
        loop = tqdm(enumerate(test_data_loader))
        for i, data in loop:
            #data = data.to(device)
            real_features, real_edge_index, real_edge_attr, batch, mol = data.features, data.edge_index.to(dtype=torch.int64), data.edge_attr, data.batch, data.mol
            mu, std = self.encode(real_features, real_edge_index, real_edge_attr, batch)
            z_reparametrized = self.reparametrize(mu, std)
            features_decoded, edges_decoded, edges_attr_decoded = self.decode(z_reparametrized)

            
            real_adj_edges, real_adj_edge_attr = batch_edge_idx_to_adj(real_edge_index, real_edge_attr, batch)

            features_loss = loss_fn(one_hot(torch.argmax(features_decoded, dim=1), num_classes = ATOM_DIM).to(torch.float64), real_features)
            edges_loss = loss_fn(edges_decoded, real_adj_edges)
            edges_attr_loss = loss_fn(one_hot(torch.argmax(edges_attr_decoded, dim=2), num_classes = BOND_DIM).to(torch.float64), real_adj_edge_attr)
            
            reconstruction_loss = features_loss + edges_loss + edges_attr_loss
            kl_div = -torch.sum(1 + torch.log(std.pow(2)) - mu.pow(2) - std.pow(2))
            loss = reconstruction_loss + kl_div
            print("Eval loss: ", loss)



In [298]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [380]:
encoder = Encoder(hidden_size=HIDDEN_SIZE)
decoder = Encoder(hidden_size=HIDDEN_SIZE)

encoder=encoder.float()
decoder = decoder.float()
g = GraphNeuralNetwork(hidden_size=512, encoder=encoder, decoder=decoder)
#g=g.float()


In [365]:
from numpy import int64
from torch.nn.functional import one_hot

def train(model, train_loader, valid_loader, l_r = 0.00001, epochs = 100):
        learning_rate = l_r

        optimizer = torch.optim.Adam(params=model.parameters(), lr= learning_rate)
        loss_fn = torch.nn.CrossEntropyLoss()
        for epoch in trange(1, epochs + 1, leave=False):
            loop = tqdm(enumerate(train_loader))
            for i, data in loop:
                data = data.to(device)

                real_features, real_edge_index, real_edge_attr, batch, mol = data.features, data.edge_index.to(dtype=torch.int64), data.edge_attr, data.batch, data.mol
                mu, std = model.encode(real_features, real_edge_index, real_edge_attr, batch)
                print(mu.shape, std.shape)
                z_reparametrized = model.reparametrize(mu, std)
                features_decoded, edges_decoded, edges_attr_decoded = model.decode(z_reparametrized)

                
                real_adj_edges, real_adj_edge_attr = batch_edge_idx_to_adjacency(real_edge_index, real_edge_attr, batch)
                #print(features_decoded.shape, real_features.shape)
                features_loss = loss_fn(one_hot(torch.argmax(features_decoded, dim=1), num_classes = ATOM_DIM).to(torch.float64), real_features)
                edges_loss = loss_fn(edges_decoded, real_adj_edges)
                edges_attr_loss = loss_fn(one_hot(torch.argmax(edges_attr_decoded, dim=2), num_classes = BOND_DIM).to(torch.float64), real_adj_edge_attr)

                reconstruction_loss = features_loss + edges_loss + edges_attr_loss
                kl_div = -torch.sum(1 + torch.log(std.pow(2)) - mu.pow(2) - std.pow(2))
                loss = reconstruction_loss + kl_div
                print(loss)
                loss.backward()
                optimizer.step() 
                optimizer.zero_grad()
                model.zero_grad()


In [301]:
from torch.nn.functional import one_hot

loop = tqdm(enumerate(val_loader))
for i, data in loop:
    if i > 0:
        break
    #print(data.features)
    real_features, real_edge_index, real_edge_attr, batch, mol = data.features, data.edge_index.to(dtype=torch.int64), data.edge_attr, data.batch, data.mol
    print(batch.shape, batch)
    print(real_features, real_edge_index, real_edge_attr)
    mu, std = g.encode(real_features, real_edge_index, real_edge_attr, batch)
                #real_features = F.one_hot(torch.argmax(real_features, dim=1), ATOM_DIM)
    z_reparametrized = g.reparametrize(mu, std)
    features_decoded, edges_decoded, edges_attr_decoded = g.decode(z_reparametrized)
    #print(features_decoded, one_hot(torch.argmax(features_decoded, dim=1), num_classes = ATOM_DIM).to(torch.float64))
    mol = graph_to_molecule((real_features, real_edge_index, real_edge_attr))
    mol = Chem.rdchem.Mol(mol)
    mol2 = graph_to_molecule((features_decoded, edges_decoded, edges_attr_decoded))
    Draw.MolToImage(mol)      # Why does it not appear, while drawing in the loop?

0it [00:00, ?it/s]

torch.Size([1024]) tensor([ 0,  0,  0,  ..., 31, 31, 31])
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.]]) tensor([[  22,   23,   24,  ..., 1004, 1014, 1006],
        [  22,   23,   24,  ..., 1009, 1006, 1014]]) tensor([[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        ...,
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])
*
 torch.Size([1024, 512])
torch.Size([512])


IndexError: index 54 is out of bounds for axis 0 with size 32

In [ ]:
print(features_decoded, edges_decoded, edges_attr_decoded)

tensor([[0.0827, 0.0828, 0.0844, 0.0812, 0.0804, 0.0788, 0.0801, 0.0875, 0.0866,
         0.0863, 0.0900, 0.0791],
        [0.0867, 0.0863, 0.0831, 0.0832, 0.0789, 0.0873, 0.0799, 0.0818, 0.0831,
         0.0893, 0.0846, 0.0757],
        [0.0832, 0.0854, 0.0793, 0.0866, 0.0837, 0.0789, 0.0788, 0.0826, 0.0878,
         0.0857, 0.0887, 0.0792],
        [0.0844, 0.0817, 0.0801, 0.0830, 0.0833, 0.0914, 0.0741, 0.0773, 0.0871,
         0.0868, 0.0923, 0.0785],
        [0.0866, 0.0849, 0.0829, 0.0827, 0.0812, 0.0835, 0.0847, 0.0808, 0.0804,
         0.0845, 0.0830, 0.0848],
        [0.0793, 0.0805, 0.0837, 0.0801, 0.0864, 0.0844, 0.0851, 0.0854, 0.0812,
         0.0887, 0.0798, 0.0853],
        [0.0816, 0.0922, 0.0829, 0.0800, 0.0893, 0.0851, 0.0839, 0.0761, 0.0818,
         0.0837, 0.0779, 0.0854],
        [0.0862, 0.0820, 0.0807, 0.0811, 0.0888, 0.0775, 0.0805, 0.0880, 0.0796,
         0.0801, 0.0856, 0.0898],
        [0.0877, 0.0782, 0.0785, 0.0787, 0.0848, 0.0868, 0.0902, 0.0811, 0.0905,

In [ ]:
Draw.MolToImage(mol2) 

In [ ]:
Draw.MolToImage(mol) 

In [381]:

train(g, train_loader=train_loader, valid_loader=test_data_loader, epochs = 30)

  0%|          | 0/30 [00:00<?, ?it/s]

0it [00:00, ?it/s]

tensor([ 0,  0,  0,  ..., 31, 31, 31])
before pooling  torch.Size([1024, 512])
after pooling  torch.Size([32, 512])
after densing  torch.Size([32, 512])
torch.Size([32, 128]) torch.Size([32, 128])


RuntimeError: shape '[32, 32]' is invalid for input of size 32768